<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 144.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 130.8 MB/s eta 0:00:00


## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [32]:
import pandas as pd

dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

print(df.head())

   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [6]:
df.dtypes

ResponseId               int64
MainBranch              object
Age                     object
Employment              object
RemoteWork              object
                        ...   
JobSatPoints_11        float64
SurveyLength            object
SurveyEase              object
ConvertedCompYearly    float64
JobSat                 float64
Length: 114, dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB


In [8]:
missing_values = df.isnull().sum()
print(missing_values)

ResponseId                 0
MainBranch                 0
Age                        0
Employment                 0
RemoteWork             10631
                       ...  
JobSatPoints_11        35992
SurveyLength            9255
SurveyEase              9199
ConvertedCompYearly    42002
JobSat                 36311
Length: 114, dtype: int64


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [9]:
df[["JobSatPoints_6", "JobSatPoints_7", "JobSatPoints_8", "JobSatPoints_9", "JobSatPoints_10", "JobSatPoints_11", "JobSat", "ConvertedCompYearly"]].describe()

,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,JobSat,ConvertedCompYearly
count,29450.000000,29448.00000,29456.000000,29456.000000,29450.000000,29445.000000,29126.000000,2.343500e+04
mean,24.343232,22.96522,20.278165,16.169432,10.955713,9.953948,6.935041,8.615529e+04
std,27.089360,27.01774,26.108110,24.845032,22.906263,21.775652,2.088259,1.867570e+05
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,6.000000,3.271200e+04
50%,20.000000,15.00000,10.000000,5.000000,0.000000,0.000000,7.000000,6.500000e+04
75%,30.000000,30.00000,25.000000,20.000000,10.000000,10.000000,8.000000,1.079715e+05
max,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,10.000000,1.625660e+07


### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [11]:
pattern = r'^[A-Za-z]+\s\d+$'
inconsistent_entries = df[~df['Country'].astype(str).str.contains(pattern, na=False)]
print("Inconsistent entries based on pattern:\n", inconsistent_entries)

Inconsistent entries based on pattern:
        ResponseId                      MainBranch                 Age  \
0               1  I am a developer by profession  Under 18 years old   
1               2  I am a developer by profession     35-44 years old   
2               3  I am a developer by profession     45-54 years old   
3               4           I am learning to code     18-24 years old   
4               5  I am a developer by profession     18-24 years old   
...           ...                             ...                 ...   
65432       65433  I am a developer by profession     18-24 years old   
65433       65434  I am a developer by profession     25-34 years old   
65434       65435  I am a developer by profession     25-34 years old   
65435       65436  I am a developer by profession     18-24 years old   
65436       65437     I code primarily as a hobby     18-24 years old   

                Employment                            RemoteWork   Check  \
0      

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [12]:
df.Country.value_counts().index

Index(['United States of America', 'Germany', 'India',
       'United Kingdom of Great Britain and Northern Ireland', 'Ukraine',
       'France', 'Canada', 'Poland', 'Netherlands', 'Brazil',
       ...
       'Niger', 'Guinea', 'Saint Kitts and Nevis', 'Lesotho', 'Haiti',
       'Micronesia, Federated States of...', 'Nauru', 'Chad', 'Djibouti',
       'Solomon Islands'],
      dtype='object', name='Country', length=185)

In [3]:
def standardize_column(df, Country, mapping):
    df[Country] = df[Country].map(mapping).fillna(df[Country])
    return df
data = {'Country': ['United States of America', 'Germany', 'India',
       'United Kingdom of Great Britain and Northern Ireland', 'Ukraine',
       'France', 'Canada', 'Poland', 'Netherlands', 'Brazil',
       'Niger', 'Guinea', 'Saint Kitts and Nevis', 'Lesotho', 'Haiti',
       'Micronesia, Federated States of...', 'Nauru', 'Chad', 'Djibouti',
       'Solomon Islands']}
df = pd.DataFrame(data)

Country_mapping = {
        'United States of America': 'USA', 
        'Germany': 'Germany', 
        'India': 'India',
        'United Kingdom of Great Britain and Northern Ireland': 'UK', 
        'Ukraine': 'Ukraine',
        'France': 'France', 
        'Canada': 'Canada',
        'Poland': 'Poland',
        'Netherlands': 'Netherlands',
        'Brazil': 'Brazil',
        'Niger': 'Niger', 
        'Guinea': 'Guinea', 
        'Saint Kitts and Nevis': 'St.Kitts/Nevis', 
        'Lesotho': 'Lesotho', 
        'Haiti': 'Haiti',
        'Micronesia, Federated States of...': 'Micronesia', 
        'Nauru': 'Nauru', 
        'Chad': 'Chad', 
        'Djibouti': 'Djibouti',
        'Solomon Islands': 'Solomon Islands'
}

df = standardize_column(df, 'Country', Country_mapping)
print(df)

            Country
0               USA
1           Germany
2             India
3                UK
4           Ukraine
5            France
6            Canada
7            Poland
8       Netherlands
9            Brazil
10            Niger
11           Guinea
12   St.Kitts/Nevis
13          Lesotho
14            Haiti
15       Micronesia
16            Nauru
17             Chad
18         Djibouti
19  Solomon Islands


In [4]:
def standardize_column(df, EdLevel, mapping):
    df[EdLevel] = df[EdLevel].map(mapping).fillna(df[EdLevel])
    return df

data = {'EdLevel': ['Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)',
       'Some college/university study without earning a degree',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Professional degree (JD, MD, Ph.D, Ed.D, etc.)',
       'Associate degree (A.A., A.S., etc.)', 'Primary/elementary school',
       'Something else']}
df = pd.DataFrame(data)

EdLevel_mapping = {
   'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 'Bachelor Degree',
       'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 'Master Degree',
       'Some college/university study without earning a degree': 'Some College/University',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 'Highschool Diploma',
       'Professional degree (JD, MD, Ph.D, Ed.D, etc.)': 'Doctorate Degree',
       'Associate degree (A.A., A.S., etc.)': 'Associate Degree',
       'Primary/elementary school': 'Primary School',
       'Something else': 'Other'
}

df = standardize_column(df, 'EdLevel', EdLevel_mapping)
print(df)

                   EdLevel
0          Bachelor Degree
1            Master Degree
2  Some College/University
3       Highschool Diploma
4         Doctorate Degree
5         Associate Degree
6           Primary School
7                    Other


### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [5]:
data = {'Employment': ['Employed, full-time',
       'Independent contractor, freelancer, or self-employed',
       'Student, full-time',
       'Employed, full-time;Independent contractor, freelancer, or self-employed',
       'Not employed, but looking for work', 'Employed, part-time',
       'Student, full-time;Employed, part-time',
       'Employed, full-time;Student, full-time',
       'Employed, full-time;Student, part-time',
       'Student, full-time;Not employed, but looking for work',
       'Employed, full-time;Student, full-time;Not employed, but looking for work;Independent contractor, freelancer, or self-employed',
       'Employed, full-time;Not employed, but looking for work;Not employed, and not looking for work;Employed, part-time',
       'Employed, full-time;Student, full-time;Independent contractor, freelancer, or self-employed;Student, part-time;Retired',
       'Employed, full-time;Student, full-time;Not employed, but looking for work;Student, part-time;Employed, part-time',
       'Employed, full-time;Student, full-time;Not employed, but looking for work;Independent contractor, freelancer, or self-employed;Student, part-time;Employed, part-time;Retired',
       'Not employed, but looking for work;Independent contractor, freelancer, or self-employed;Not employed, and not looking for work;Employed, part-time',
       'Student, full-time;Retired',
       'Employed, full-time;Not employed, but looking for work;Student, part-time',
       'Not employed, and not looking for work;Student, part-time;Employed, part-time',
       'Not employed, but looking for work;Independent contractor, freelancer, or self-employed;Student, part-time;Retired'
    
]}
df = pd.DataFrame(data)

df_encoded = pd.get_dummies(df, columns=['Employment'])

print(df_encoded)

    Employment_Employed, full-time  \
0                             True   
1                            False   
2                            False   
3                            False   
4                            False   
5                            False   
6                            False   
7                            False   
8                            False   
9                            False   
10                           False   
11                           False   
12                           False   
13                           False   
14                           False   
15                           False   
16                           False   
17                           False   
18                           False   
19                           False   

    Employment_Employed, full-time;Independent contractor, freelancer, or self-employed  \
0                                               False                                     
1                  

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [24]:
def find_missing_values(df):
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    return missing_values.sort_values(ascending=False)

missing_counts = find_missing_values(df)
print(missing_counts)

AINextMuch less integrated    64289
AINextLess integrated         63082
AINextNo change               52939
AINextMuch more integrated    51999
EmbeddedAdmired               48704
                              ...  
YearsCode                      5568
NEWSOSites                     5151
LearnCode                      4949
EdLevel                        4653
AISelect                       4530
Length: 108, dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [25]:
column_name = 'ConvertedCompYearly'
missing_count = df['ConvertedCompYearly'].isnull().sum()
print(f"Number of missing values in column '{'ConvertedCompYearly'}': {missing_count}")

Number of missing values in column 'ConvertedCompYearly': 0


In [11]:
df['ConvertedCompYearly'].describe()

count    2.343500e+04
mean     8.615529e+04
std      1.867570e+05
min      1.000000e+00
25%      3.271200e+04
50%      6.500000e+04
75%      1.079715e+05
max      1.625660e+07
Name: ConvertedCompYearly, dtype: float64

<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [26]:
options = ['Hybrid (some remote, some in-person)', 'Remote', 'In-person']

most_freq = pd.Series(options)
most_freq_counts = most_freq.value_counts()
print("Most frequent value is", most_freq_counts.index[0])

def impute_with_mode(df, RemoteWork):
    mode_value = df['RemoteWork'].mode()[0]
    df.fillna({'RemoteWork': mode_value}, inplace=True)
    return df

data = {'RemoteWork': ['Hybrid (some remote, some in-person)', 'Remote', 'In-person']}
df = pd.DataFrame(data)

df = impute_with_mode(df, 'RemoteWork')
print(df)


Most frequent value is Hybrid (some remote, some in-person)
                             RemoteWork
0  Hybrid (some remote, some in-person)
1                                Remote
2                             In-person


### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [27]:
def min_max_scale_column(df, ConvertedCompYearly):
    min_value = df['ConvertedCompYearly'].min()
    max_value = df['ConvertedCompYearly'].max()
    df['ConvertedCompYearly_MinMax'] = (df['ConvertedCompYearly'] - min_value) / (max_value - min_value)
    return df

data = {'ConvertedCompYearly': [2.343500e+04, 8.615529e+04, 1.867570e+05, 1.000000e+00, 3.271200e+04, 6.500000e+04, 1.079715e+05, 1.625660e+07]}
4,
df = pd.DataFrame(data)

df = min_max_scale_column(df, 'ConvertedCompYearly')
print(df)

   ConvertedCompYearly  ConvertedCompYearly_MinMax
0             23435.00                    0.001442
1             86155.29                    0.005300
2            186757.00                    0.011488
3                 1.00                    0.000000
4             32712.00                    0.002012
5             65000.00                    0.003998
6            107971.50                    0.006642
7          16256600.00                    1.000000


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [28]:
df['ConvertedCompYearly'].value_counts().index

Index([23435.0, 86155.29, 186757.0, 1.0, 32712.0, 65000.0, 107971.5,
       16256600.0],
      dtype='float64', name='ConvertedCompYearly')

In [30]:
import numpy as np

data = {'ConvertedCompYearly': [23435.0, 86155.29, 186757.0, 1.0, 32712.0, 65000.0, 107971.5,
       16256600.0]}
df = pd.DataFrame(data)

if (df['ConvertedCompYearly'] <= 0).any():
    df['ConvertedCompYearly'] = df['ConvertedCompYearly'] + 1

df['ConvertedCompYearly_log'] = np.log(df['ConvertedCompYearly'])

print("Skewness before:", df['ConvertedCompYearly'].skew())
print("Skewness after:", df['ConvertedCompYearly_log'].skew())

print(df)

Skewness before: 2.827840917819262
Skewness after: -1.6468299300535483
   ConvertedCompYearly  ConvertedCompYearly_log
0             23435.00                10.061986
1             86155.29                11.363907
2            186757.00                12.137564
3                 1.00                 0.000000
4             32712.00                10.395497
5             65000.00                11.082143
6            107971.50                11.589623
7          16256600.00                16.604010


### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [33]:
df.YearsCodePro.value_counts().index

Index(['2', '3', '5', '10', '4', 'Less than 1 year', '6', '1', '8', '7', '12',
       '15', '20', '9', '11', '13', '14', '25', '16', '18', '17', '30', '24',
       '19', '22', '23', '26', '27', '21', '28', '35', '29', '40', '32', '34',
       '38', '33', '36', '31', '37', '45', '42', '39', '41',
       'More than 50 years', '44', '43', '46', '50', '48', '49', '47'],
      dtype='object', name='YearsCodePro')

In [42]:
df['ExperienceLevel'].value_counts().index

Index(['2', '3', '5', '10', '4', 'Less than 1 year', '6', '1', '8', '7', '12',
       '15', '20', '9', '11', '13', '14', '25', '16', '18', '17', '30', '24',
       '19', '22', '23', '26', '27', '21', '28', '35', '29', '40', '32', '34',
       '38', '33', '36', '31', '37', '45', '42', '39', '41',
       'More than 50 years', '44', '43', '46', '50', '48', '49', '47'],
      dtype='object', name='ExperienceLevel')

In [41]:
df['ExperienceLevel'] = df['YearsCodePro']
print(df)

          YearsCodePro     ExperienceLevel
0                    2                   2
1                    3                   3
2                    5                   5
3                   10                  10
4                    4                   4
5     Less than 1 year    Less than 1 year
6                    6                   6
7                    1                   1
8                    8                   8
9                    7                   7
10                  12                  12
11                  15                  15
12                  20                  20
13                   9                   9
14                  11                  11
15                  13                  13
16                  14                  14
17                  25                  25
18                  16                  16
19                  18                  18
20                  17                  17
21                  30                  30
22         

### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
